In [1]:
import xarray as xr
import reverse_geocoder as rg
from tqdm import tqdm
import pandas as pd
import itertools
import numpy as np
import pandas as pd
from scipy.spatial import distance
import matplotlib.pyplot as plt
import pickle as pkl

In [84]:
country_size={"DE":357600,"AT":83879,"SE":450295,"FR":543941,"IT":301958,"PL":312679,"ES":498485,"NL":41543,"BE":30528, "IE":70273}
geocoding=pkl.load(open("country_dict.pkl","rb"))
data=xr.open_dataset(filename_or_obj=r"C:\\Daten\\Foschung\\RiskAware\\Data\\Weather\\rainfall\\gpcp_v01r03_daily_d20220101.nc", engine='netcdf4')
latitudes=data.latitude.values
longitudes=data.longitude.values

In [78]:
dates=pd.date_range(start="2022-01-01",end="2022-12-31",freq="D")
country_precip=pd.DataFrame(0.0,index=dates,columns=country_size.keys())
for date in dates:
    day=date.strftime("%Y%m%d")
    data= xr.open_dataset(filename_or_obj=r"C:\\Daten\\Foschung\\RiskAware\\Data\\Weather\\rainfall\\gpcp_v01r03_daily_d"+day+".nc", engine='netcdf4')
    precip=pd.DataFrame(np.array(data.precip).reshape(37,39),index=latitudes,columns=longitudes)
    precip[precip<0]=0
    for country in country_size.keys():
        country_keys = [k for k,v in geocoding.items() if v==country]
        for key in country_keys:
            if key[1]<0:
                key=(key[0],key[1]+360)
            country_precip.loc[day,country] += precip.loc[key]*np.cos(key[0])*pow(111320,2)*(country_size[country]/(len(country_keys)*np.cos(key[0])*pow(111.320,2)))


In [85]:
#Datacenter Location Coordinates
datacenter_loc={"DE":(50.110924,8.682127),"FR":(48.856667,2.351667),"IE":(51.509865,-0.118092),"IT":(45.4625,9.1864),
                "NL":(52.3702,4.8904),"PL":(52.2167,21.0333),"ES":(40.412,-3.7039),
                "SE":(59.325,18.05),"BE":(50.8466,4.3517),"AT":(48.2083,16.3731)}
#Country Size in KM2
country_size={"DE":357600,"AT":83879,"SE":450295,"FR":543941,"IT":301958,"PL":312679,"ES":498485,"NL":41543,"BE":30528, "IE":70273}
#Daily Precipitation by 1X1 Degree Grid adjusted for Size of Country
country_precip=pd.read_csv("C:\\Daten\\Foschung\\RiskAware\\Data\\Weather\\country_precip.csv",index_col=0,parse_dates=True)

yearly_precip={"DE":612.51,"FR":645.1,"IT":601.08,"IE":1152.55,"NL":821,"PL":541.2,"ES":551.43,"SE":604.3,"BE":749.38,"AT":948.84}


Drought Indice

In [86]:
df_test = pd.read_csv('C:\Daten\Foschung\RiskAware\Code\Data\Drought\drought_geolocated.csv',index_col=0)
df_test.reset_index(drop=True, inplace=True)
df_test['cdi'] = df_test['cdi'].apply(lambda x: np.fromstring(x[1:-1], sep=' ',dtype=np.int8))
df_test = df_test[df_test['cdi'].apply(lambda x: not np.all(x == np.int8(8)))]
risk=pd.DataFrame(index=range(0,36),columns=datacenter_loc.keys())
for country in tqdm(datacenter_loc.keys()):
    # Filter for Germany
    df_country = df_test[df_test['country'] == country].copy()
    # Build the drought data for Germany
    df_country['cdi'] = df_country['cdi'].apply(lambda x: np.where(x == 6, 1, np.where(x == 5, 2, np.where(x == 4, 3, x))))

    # Calculate distances and weights
    df_country['distance'] = df_country.apply(lambda row: distance.euclidean((row['lat'], row['lon']), (datacenter_loc[country][0],datacenter_loc[country][1])), axis=1)
    df_country['weight'] = np.pow(1 / df_country['distance'],1)
    df_country['weight'] /= df_country['weight'].sum()

    # Calculate weighted drought risk
    risk_country = [np.sum(df_country['cdi'].apply(lambda x: x[i]) * df_country['weight']) for i in range(len(df_country['cdi'].iloc[0]))]

    risk.loc[:,country] = risk_country

100%|██████████| 10/10 [00:04<00:00,  2.29it/s]


In [95]:
yearly_precip_df = pd.DataFrame(np.array(yearly_precip.values()).reshape(1,10), index=[2022], columns=list(yearly_precip.keys()))

ValueError: cannot reshape array of size 1 into shape (1,10)

In [89]:
yearly_precip_df

,Country,Precipitation
0,DE,612.51
1,FR,645.10
2,IT,601.08
3,IE,1152.55
4,NL,821.00
5,PL,541.20
6,ES,551.43
7,SE,604.30
8,BE,749.38
9,AT,948.84


In [71]:
# Use the average precip for a year to scale the drought risk
risk = risk * yearly_precip

,DE,FR,GB,IT,NL,PL,ES,SE,BE,AT
0,0.170051,0.075135,0.241778,0.4374,0.002684,0.263505,0.569108,0.56592,0.004893,0.119148
1,0.113971,0.095378,0.590935,0.348324,0.004026,0.078795,0.606562,0.72141,0.006271,0.094313
2,0.150744,0.130694,0.948372,0.40459,0.035171,0.065433,0.633651,1.298952,0.007656,0.087464
3,0.27596,0.429869,1.604265,0.979195,0.105514,0.060692,1.276639,0.925463,0.024732,0.053956
4,0.110951,1.012655,1.495742,1.561965,0.001639,0.057302,1.741262,1.718377,0.030763,0.068564
5,0.123592,0.172432,0.617053,1.123598,0.00192,0.058729,1.51739,0.291534,0.031151,0.083562
6,0.090975,0.603887,0.375668,1.273376,0.0,0.03922,1.744483,0.129544,0.039823,0.294064
7,0.458468,1.717394,0.512473,1.562353,1.501459,0.290203,1.5588,0.242019,1.414878,1.162768
8,0.846881,0.907629,0.378007,1.527059,1.957381,1.282003,1.224207,1.069449,1.711878,1.674383
9,1.27506,1.095517,0.74823,1.756627,2.374,2.127858,1.114293,1.598643,1.981441,2.389707


In [55]:
risk

{'DE': [np.float64(0.17005116478008211),
  np.float64(0.11397113853802249),
  np.float64(0.15074431460206383),
  np.float64(0.2759599396174929),
  np.float64(0.11095081283701204),
  np.float64(0.12359214927360812),
  np.float64(0.09097549826243098),
  np.float64(0.4584682870816383),
  np.float64(0.8468812071043588),
  np.float64(1.2750602230964942),
  np.float64(1.035328571064977),
  np.float64(0.4045861632696324),
  np.float64(0.7231292672071259),
  np.float64(0.9477176546495968),
  np.float64(1.0970128619290522),
  np.float64(1.4741520913608288),
  np.float64(1.1880801410952135),
  np.float64(1.4859614834720145),
  np.float64(1.6680036129538078),
  np.float64(1.6990942449777924),
  np.float64(1.9045370216555166),
  np.float64(1.9511132390116552),
  np.float64(2.0566526652766717),
  np.float64(2.1613858574867826),
  np.float64(2.1721890385650253),
  np.float64(2.228644202727355),
  np.float64(2.110660773238114),
  np.float64(1.8981561122014998),
  np.float64(1.4284366421244488),
  np.